In [8]:
from collections import defaultdict

import gymnasium as gym
import numpy as np

In [2]:
gym.__version__

'0.29.1'

In [3]:
env = gym.make('Blackjack-v1', natural=False, sab=False, render_mode='rgb_array')

In [4]:
env.action_space.sample()

0

# 作业一：将离线改成在线

In [10]:

episode = 10000
Q_table = defaultdict(lambda: [0, 0])  # 0th, action-0, state: value, action-1, state: value

win_or_loss = []


def train(Q_table: defaultdict[lambda: [0, 0]], trajectory: (int, int, int)):
    gamma = 0.99
    s, a, r = trajectory
    if not Q_table[s][a]:
        Q_table[s][a] = 0
    old_G = Q_table[s][a]
    Q_table[s][a] = np.mean([old_G, gamma * r])


def policy(Q_table, state):
    if Q_table[state]:
        return np.argmax(Q_table[state])
    else:
        return env.action_space.sample()


for _ in range(episode):
    state, info = env.reset()
    while True:
        action = policy(Q_table, state)
        state_next, reward, terminated, truncated, info = env.step(action)
        # (s, a, r)
        trajectory = (state, action, reward)

        state = state_next

        if terminated or truncated:
            train(Q_table, trajectory)
            if reward == 1:
                win_or_loss.append(1)
            else:
                win_or_loss.append(0)
            break
env.close()
# %%

print(sum(win_or_loss) / len(win_or_loss))

0.4102


在线得方式会实时更新计算得Q值，模型最开始很差，但是会逐渐一点一点得变好。速度会快，但是批量off-line得更新方式因为计算mean得时候会更准确，精度更高

### response - 01

是的，你的代码没问题，而且对这个方法的问题也理解的没有问题。 

我们可以对代码做一点点的小小的变化，让它不至于刚开始的时候非常执着于错误的方向：

```python 

def policy(Q_table, state):
    eps = 0.05
    if random.random() < eps or state not in Q_table: 
        return env.action_space.sample()
    else:
        return np.argmax(Q_table[state])
    
这个在RL中是一个非常典型的问题，叫做"exploration vs exploitation" 

# 作业二


In [ ]:
我们可以修改得地方
1. 修改奖励机制，对r的值进行变化
2. 调整折扣因子gamma
3. 增加训练次数和样本量episode得值
4. 调整policy策略

In [ ]:
可以增加剩余卡牌队中高点数和低点数的比例可以进行学习

### response-02

以上思路没有问题，不过如果你想知道具体哪种方法更有效，可以自己写代码实现以下，观察一下真实的结果~ 

# 作业三

强化学习是没有绝对的对错，只是一种不同选择带来不同的后果中进行学习【具有反馈机制】，但是监督学习是在有明确的答案基础上，向标准答案学习的方式。
非监督学习是自己找出标准答案，发现数据的特征。

### response-03

强化学习其实也有“决定的对错“，例如在某个state下，是存在一个最优动作的，但是ve'ge